https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea

In [3]:
! pip install sentence-transformers

In [4]:
import pandas as pd
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
import nltk
from nltk.corpus import words
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('words')
nltk.download('wordnet')
nltk.download('stopwords')

stopwords = set(nltk.corpus.stopwords.words('spanish'))
include_stopwords = {'mas', 'menos', 'null', 'none'}
stopwords |= include_stopwords
print("Total spanish stopwords: {}\n".format(len(stopwords)))

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Total spanish stopwords: 317



In [6]:
doc = """
         Rol

    Científico(a) de Datos Senior

Descripcion

    ¡Buscamos a las personas adecuadas, personas que deseen innovar, crear, liderar, transformar! Queremos atraerte, pero más allá de eso queremos contar con el mejor talento, estamos comprometidos con el desarrollo profesional de cada uno de nuestros colaboradores. Queremos que experimentes los desafíos, recompensas y oportunidades de trabajar con la mejor empresa de telecomunicaciones del país ¿Te atreves?

Requisitos

    Profesional en Economía, Estadística, Matemáticas, Ingeniería Industrial, Sistemas, Electrónica, Telecomunicaciones o carreras afines.
    Maestría en economía, estadística o equivalentes.
    Experiencia mínimo de cinco (5) años en manejo avanzado de R / Python, Machine Learning.
    Construcción de Modelos Analíticos o Geo Espaciales.
    Indispensable experiencia en Azure Databricks y Machine Learning.
    Análisis descriptivos, modelos de clasificación, Cluster, arboles de decisión, manejo de Jupyter, TensorFlow, Keras.

Condiciones Laborales

    Lugar de Trabajo: Bogotá.
    Tipo de Contrato: A Término Indefinido.
    Salario: A convenir de acuerdo a la experiencia.

Esta vacante es divulgada a través de ticjob.co
      """

In [7]:
n_gram_range = (2, 2)
#stop_words = "spanish"
#my_stop_words = [lemma(t) for t in stopwords.words('spanish')]

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stopwords).fit([doc])
candidates = count.get_feature_names()

In [8]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [9]:
top_n = 15
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [10]:
keywords

['requisitos profesional',
 'allá queremos',
 'descriptivos modelos',
 'rol científico',
 'modelos analíticos',
 'lugar trabajo',
 'descripcion buscamos',
 'modelos clasificación',
 'cada colaboradores',
 'oportunidades trabajar',
 'construcción modelos',
 'transformar queremos',
 'buscamos personas',
 'recompensas oportunidades',
 'colaboradores queremos']

In [11]:
def max_sum_sim(doc_embedding, word_embeddings, words, top_n, nr_candidates):
    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # Get top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [12]:
max_sum_sim(doc_embedding,candidate_embeddings,candidates,top_n=15,nr_candidates=15)

['requisitos profesional',
 'allá queremos',
 'descriptivos modelos',
 'rol científico',
 'modelos analíticos',
 'lugar trabajo',
 'descripcion buscamos',
 'modelos clasificación',
 'cada colaboradores',
 'oportunidades trabajar',
 'construcción modelos',
 'transformar queremos',
 'buscamos personas',
 'recompensas oportunidades',
 'colaboradores queremos']

In [13]:
def mmr(doc_embedding, word_embeddings, words, top_n, diversity):

    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [14]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n = 15, diversity = 1)

['colaboradores queremos',
 'machine learning',
 'senior descripcion',
 'avanzado python',
 'analíticos geo',
 'azure databricks',
 'indispensable experiencia',
 'trabajo bogotá',
 'sistemas electrónica',
 'descripcion buscamos',
 'jupyter tensorflow',
 'economía estadística',
 'experiencia mínimo',
 'condiciones laborales',
 'mejor talento']

In [15]:
def keywords(doc,n_gram_range,top_n):
  #n_gram_range = (2, 2)

  # Extract candidate words/phrases
  try:
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stopwords).fit([doc])
    candidates = count.get_feature_names()

    model = SentenceTransformer('distilbert-base-nli-mean-tokens')
    doc_embedding = model.encode([doc])
    candidate_embeddings = model.encode(candidates)

    #top_n = 15
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
  
  except:
    keywords = []

  return keywords

In [16]:
keywords(doc,n_gram_range = (3, 3),top_n=15)

['atraerte allá queremos',
 'comprometidos desarrollo profesional',
 'liderar transformar queremos',
 'desafíos recompensas oportunidades',
 'descripcion buscamos personas',
 'construcción modelos analíticos',
 'cada colaboradores queremos',
 'allá queremos contar',
 'colaboradores queremos experimentes',
 'oportunidades trabajar mejor',
 'laborales lugar trabajo',
 'talento comprometidos desarrollo',
 'recompensas oportunidades trabajar',
 'profesional cada colaboradores',
 'buscamos personas adecuadas']

In [17]:
def max_sum_sim_iter(doc, n_gram_range, top_n, nr_candidates):

    try:

      #n_gram_range = (2, 2)
      #stop_words = "spanish"
      #my_stop_words = [lemma(t) for t in stopwords.words('spanish')]

      # Extract candidate words/phrases
      count = CountVectorizer(ngram_range=n_gram_range, stop_words=stopwords).fit([doc])
      candidates = count.get_feature_names()

      model = SentenceTransformer('distilbert-base-nli-mean-tokens')
      doc_embedding = model.encode([doc])
      candidate_embeddings = model.encode(candidates)

      # Calculate distances and extract keywords
      distances = cosine_similarity(doc_embedding, candidate_embeddings)
      distances_candidates = cosine_similarity(candidate_embeddings, 
                                              candidate_embeddings)

      # Get top_n words as candidates based on cosine similarity
      words_idx = list(distances.argsort()[0][-nr_candidates:])
      words_vals = [candidates[index] for index in words_idx]
      distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

      # Calculate the combination of words that are the least similar to each other
      min_sim = np.inf
      candidate = None
      for combination in itertools.combinations(range(len(words_idx)), top_n):
          sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
          if sim < min_sim:
              candidate = combination
              min_sim = sim
      try:
        result = [words_vals[idx] for idx in candidate]
      except:
        result = []
    
    except:
      result = []
    
    return result


In [18]:
max_sum_sim_iter(doc, n_gram_range = (3, 3),top_n=15,nr_candidates=15)

['atraerte allá queremos',
 'comprometidos desarrollo profesional',
 'liderar transformar queremos',
 'desafíos recompensas oportunidades',
 'descripcion buscamos personas',
 'construcción modelos analíticos',
 'cada colaboradores queremos',
 'allá queremos contar',
 'colaboradores queremos experimentes',
 'oportunidades trabajar mejor',
 'laborales lugar trabajo',
 'talento comprometidos desarrollo',
 'recompensas oportunidades trabajar',
 'profesional cada colaboradores',
 'buscamos personas adecuadas']

In [19]:
def mmr_iter(doc, n_gram_range, top_n, diversity):

    try:
      #n_gram_range = (2, 2)
      #stop_words = "spanish"
      #my_stop_words = [lemma(t) for t in stopwords.words('spanish')]

      # Extract candidate words/phrases
      count = CountVectorizer(ngram_range=n_gram_range, stop_words=stopwords).fit([doc])
      candidates = count.get_feature_names()

      model = SentenceTransformer('distilbert-base-nli-mean-tokens')
      doc_embedding = model.encode([doc])
      candidate_embeddings = model.encode(candidates)

      # Extract similarity within words, and between words and the document
      word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)
      word_similarity = cosine_similarity(candidate_embeddings)

      # Initialize candidates and already choose best keyword/keyphras
      try:
        keywords_idx = [np.argmax(word_doc_similarity)]
      except:
        keywords_idx = []
      candidates_idx = [i for i in range(len(candidates)) if i != keywords_idx[0]]

      for _ in range(top_n - 1):
          # Extract similarities within candidates and
          # between candidates and selected keywords/phrases
          candidate_similarities = word_doc_similarity[candidates_idx, :]
          try:
            target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)
          except:
            target_similarities = np.empty((1,1),dtype=object)

          # Calculate MMR
          try:
            mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
          except:
            mmr = np.empty((1,1),dtype=object)
          try:
            mmr_idx = candidates_idx[np.argmax(mmr)]
          except:
            mmr_idx = None

          # Update keywords & candidates
          keywords_idx.append(mmr_idx)
          try:
            candidates_idx.remove(mmr_idx)
          except:
            continue

      try:
        result = [candidates[idx] for idx in keywords_idx]
      except:
        result = []

    except:
      result = []

    return result

In [20]:
mmr_iter(doc,n_gram_range = (3,3),top_n = 15, diversity = 1)

['buscamos personas adecuadas',
 'python machine learning',
 'indispensable experiencia azure',
 'matemáticas ingeniería industrial',
 'datos senior descripcion',
 'bogotá tipo contrato',
 'learning análisis descriptivos',
 'manejo avanzado python',
 'contar mejor talento',
 'modelos analíticos geo',
 'manejo jupyter tensorflow',
 'azure databricks machine',
 'maestría economía estadística',
 'vacante divulgada través',
 'crear liderar transformar']

In [21]:
category_model = pd.read_excel('/content/category_model.xlsx')
category_model

,Unnamed: 0,categoria,prediction_with_category_model,grouped_doc
0,20,administracion_oficina,0,auxiliar i atencion integral clientes ¡unete t...
1,406,administracion_oficina,1,prueba nov asdasd gerente regional cav s geren...
2,14,administracion_oficina,2,scrum master liderar guiar equipos desarrollo ...
3,13,administracion_oficina,3,analista i estructura contratacion ¡unete tale...
4,943,bodega_logistica_y_transporte,0,supervisor produccion oferta empleo buscamos p...
...,...,...,...,...
83,0,software_informatica_y_telecomunicaciones,3,analista ii desarrollo mantenimiento ¡unete ta...
84,423,ventas,0,promotor tecnologia riohacha importante compan...
85,71,ventas,1,consultor inversiones barranquilla ¡unete tale...
86,467,ventas,2,director tienda persona experiencia manejo per...


In [22]:
general_model = pd.read_excel('/content/general_model.xlsx')
general_model

,Unnamed: 0,categoria,prediction_with_general_model,grouped_doc
0,16,administracion_oficina,0,analista planeacion organizacional hdi seguros...
1,13,administracion_oficina,1,analista i estructura contratacion ¡unete tale...
2,407,administracion_oficina,2,analistas comercial itpc ccv analistas comerci...
3,406,administracion_oficina,3,prueba nov asdasd gerente regional cav s geren...
4,750,bodega_logistica_y_transporte,0,domiciliario funza mosquera madrid requiere ho...
...,...,...,...,...
83,112,software_informatica_y_telecomunicaciones,3,ingeniero operacion crm ingeniero operacion cr...
84,71,ventas,0,consultor inversiones barranquilla ¡unete tale...
85,426,ventas,1,ejecutivo comercial experiencia venta llantas ...
86,501,ventas,2,agente seguros juchitan integrate equipo traba...


In [23]:
df_category_model = category_model.copy()
#df_category_model['keywords'] = df_category_model['grouped_doc'].apply(lambda x: keywords(x,(2,2),15))

In [ ]:
df_category_model['max_sum_sim_iter'] = df_category_model['grouped_doc'].apply(lambda x: max_sum_sim_iter(x,n_gram_range = (2,2),top_n=15,nr_candidates=15))
df_category_model

In [24]:
df_category_model['mmr_iter'] = df_category_model['grouped_doc'].apply(lambda x: mmr_iter(x,n_gram_range = (3,3),top_n = 15, diversity = 1))
df_category_model

,Unnamed: 0,categoria,prediction_with_category_model,grouped_doc,mmr_iter
0,20,administracion_oficina,0,auxiliar i atencion integral clientes ¡unete t...,"[concesionarios clientes independientes, rifle..."
1,406,administracion_oficina,1,prueba nov asdasd gerente regional cav s geren...,"[induccion gracias daviplata, nov asdasd geren..."
2,14,administracion_oficina,2,scrum master liderar guiar equipos desarrollo ...,"[seguros colombia buscando, minimo producto vi..."
3,13,administracion_oficina,3,analista i estructura contratacion ¡unete tale...,"[indefinido analista operaciones, for microsof..."
4,943,bodega_logistica_y_transporte,0,supervisor produccion oferta empleo buscamos p...,"[supervisor produccion oferta, cliente vacunas..."
...,...,...,...,...,...
83,0,software_informatica_y_telecomunicaciones,3,analista ii desarrollo mantenimiento ¡unete ta...,"[programacion java oracle, fines experiencia l..."
84,423,ventas,0,promotor tecnologia riohacha importante compan...,"[promotor tecnologia riohacha, retail call cen..."
85,71,ventas,1,consultor inversiones barranquilla ¡unete tale...,"[impulsadoras bachilleres experiencia, america..."
86,467,ventas,2,director tienda persona experiencia manejo per...,"[prinicipales responsabilidades liderar, great..."


In [25]:
df_category_model.to_excel('df_category_model_diversity1.xlsx',encoding='utf-8-sig',index = True)

General model

In [26]:
df_general_model = general_model.copy()
#df_general_model['keywords'] = df_general_model['grouped_doc'].apply(lambda x: keywords(x,(3,3),15))
df_general_model

,Unnamed: 0,categoria,prediction_with_general_model,grouped_doc
0,16,administracion_oficina,0,analista planeacion organizacional hdi seguros...
1,13,administracion_oficina,1,analista i estructura contratacion ¡unete tale...
2,407,administracion_oficina,2,analistas comercial itpc ccv analistas comerci...
3,406,administracion_oficina,3,prueba nov asdasd gerente regional cav s geren...
4,750,bodega_logistica_y_transporte,0,domiciliario funza mosquera madrid requiere ho...
...,...,...,...,...
83,112,software_informatica_y_telecomunicaciones,3,ingeniero operacion crm ingeniero operacion cr...
84,71,ventas,0,consultor inversiones barranquilla ¡unete tale...
85,426,ventas,1,ejecutivo comercial experiencia venta llantas ...
86,501,ventas,2,agente seguros juchitan integrate equipo traba...


In [ ]:
df_general_model['max_sum_sim_iter'] = df_general_model['grouped_doc'].apply(lambda x: max_sum_sim_iter(x,n_gram_range = (3,3),top_n=15,nr_candidates=15))
df_general_model

In [27]:
df_general_model['mmr_iter'] = df_general_model['grouped_doc'].apply(lambda x: mmr_iter(x,n_gram_range = (3,3),top_n = 15, diversity = 1))
df_general_model

,Unnamed: 0,categoria,prediction_with_general_model,grouped_doc,mmr_iter
0,16,administracion_oficina,0,analista planeacion organizacional hdi seguros...,"[seguros colombia buscando, control gtr fines,..."
1,13,administracion_oficina,1,analista i estructura contratacion ¡unete tale...,"[constructoras inmobiliarias cajas, rifle amer..."
2,407,administracion_oficina,2,analistas comercial itpc ccv analistas comerci...,"[comercial analistas comercial, bayport busca ..."
3,406,administracion_oficina,3,prueba nov asdasd gerente regional cav s geren...,"[induccion gracias daviplata, nov asdasd geren..."
4,750,bodega_logistica_y_transporte,0,domiciliario funza mosquera madrid requiere ho...,"[mosquera madrid requiere, vida hojasdevida ma..."
...,...,...,...,...,...
83,112,software_informatica_y_telecomunicaciones,3,ingeniero operacion crm ingeniero operacion cr...,"[google analytics salesforce, afin responsable..."
84,71,ventas,0,consultor inversiones barranquilla ¡unete tale...,"[comerciales igual impulsar, google analytics ..."
85,426,ventas,1,ejecutivo comercial experiencia venta llantas ...,"[baterias ejecutivo comercial, american eagle ..."
86,501,ventas,2,agente seguros juchitan integrate equipo traba...,"[seguros metlife cdmx, full time medio, requie..."


In [29]:
df_general_model.to_excel('df_general_model_diversity1.xlsx',encoding='utf-8-sig',index = True)